In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.environ['KAGGLE_USERNAME'] = "lampe01" # username from the json file
os.environ['KAGGLE_KEY'] = "0fe21bb583c2cd3230e9d928c96e27f7" # key from the json file
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/CirCor"

In [3]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6085 sha256=1cb5f74839c3c091cd379b26514e24cbb9d970843737cfa3089e8d9cbd3b0898
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [5]:
pip install Ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.3 MB/s 


In [6]:
pip uninstall scikit-learn

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-1.0.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-1.0.2


In [7]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.8 MB 52.6 MB/s 


In [8]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
pip install pyAudioProcessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.8 MB/s 
     |████████████████████████████████| 11.5 MB 30.8 MB/s 
     |████████████████████████████████| 25.9 MB 1.5 MB/s 
     |████████████████████████████████| 280 kB 41.2 MB/s 
     |████████████████████████████████| 22.3 MB 63.9 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
  Attempting uninstall: pytest
    Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninsta

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import itertools
import os
import sys
import pywt
import librosa
import sklearn
import scipy
import scipy.io
import scipy.io.wavfile
from scipy import signal
from scipy.stats import kurtosis, skew, tstd
from scipy.signal import periodogram, welch, butter, lfilter, freqz
from os import path
from pydub import AudioSegment
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, Dropout, AlphaDropout, Activation, Flatten, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping,ReduceLROnPlateau,ProgbarLogger
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, recall_score, classification_report, auc, roc_curve, precision_recall_fscore_support, f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from pyAudioProcessing.extract_features import get_features

In [11]:
mdata = pd.read_csv('/content/drive/MyDrive/ReCirCor/training_data.csv')
input_dir = '/content/drive/MyDrive/ReCirCor/training_data/training_data'

In [12]:
#Gets all the txt files
def getfilenames(inputdir):
    Fname = list()
    Fname = sorted([fname for fname in os.listdir(inputdir) if not fname.startswith('.')and fname.endswith('.txt')])
    
    return Fname

In [13]:
#List of Labels - 0 for normal, 1 for murmur
labels = [0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,1,1,1]
len(labels)

23

In [14]:
#They were originally named differently and I have renamed and numbered them for ease of access.
#The michigan heart sound recordings are in mp3 format
#The recordings are converted to wav format for easy processing as most python libraries require the audio format to be wav.

def converttowav(filepath):
  destinationpath = filepath.split('.')[0] + '.wav'
  sound = AudioSegment.from_mp3(filepath)
  sound.export(destinationpath, format = "wav")

filepathslist = list()
for i in range(23):
  filepath = str(i+1) + '.mp3'
  converttowav(filepath)
  filepathslist.append(str(i+1)+'.wav')

In [15]:
#Creating an empty dataframe with columns to store filenames and the respective labels
dataframe = pd.DataFrame(columns = ['filename', 'labels'])

In [16]:
filepathslist

['1.wav',
 '2.wav',
 '3.wav',
 '4.wav',
 '5.wav',
 '6.wav',
 '7.wav',
 '8.wav',
 '9.wav',
 '10.wav',
 '11.wav',
 '12.wav',
 '13.wav',
 '14.wav',
 '15.wav',
 '16.wav',
 '17.wav',
 '18.wav',
 '19.wav',
 '20.wav',
 '21.wav',
 '22.wav',
 '23.wav']

In [17]:
labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]

In [18]:
labels[18]

0

In [19]:
#Inserting filename and label values into dataframe
dataframe['filename'] = filepathslist
dataframe['labels'] = labels

In [20]:
dataframenew = pd.DataFrame(columns = ['new ID', 'wav filename', 'labels'])

In [21]:
import random

In [22]:
newID = list()
wavfilename = list()
newlabels = list()
for i in range(250):
  x = random.randint(1,23)
  newID.append(str(i+1))
  wavfilename.append(str(x)+'.wav')
  newlabels.append(labels[x-1])

In [23]:
#Get only murmur recordings
def getrecmur(inputdir):
    fname = getfilenames(inputdir)
    items = []
    rinfo = []
    recf = []
    labels = []
    recfile = []
    for i in fname:
        i = inputdir + '/' + i
        with open(i, 'r') as f:
            content = f.readlines()
        numloc = int(content[0].split(' ')[1])
        for i in range(numloc):
            for p in range(len(content)):
                t = content[p].split('\n')
                for it in t:
                    if it.startswith('#Murmur:'):
                        tar = it.split(': ')[1]
                        if tar == 'Present':
                          recinfo = content[i+1].split(' ')
                          rinfo.append(recinfo)
                          l = 1
                          labels.append(l)
                        elif tar == 'Absent':
                          recinfo = content[i+1].split(' ')
                          rinfo.append(recinfo)
                          l = 0
                          labels.append(l)
    for items in rinfo:
        for i in range(len(items)):
            if items[i].endswith('.wav'):
                recfile = input_dir + '/' + items[i]
        recf.append(recfile)
    return rinfo, recf, labels

rin, ref, label = getrecmur(input_dir)

In [24]:
len(rin)

3007

In [25]:
ref

['/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_AV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_PV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_TV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_MV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_AV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_PV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_TV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_MV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14998_AV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14998_PV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14998_TV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14998_MV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/23625_AV.wav',

In [26]:
x = 250
for i in range(100):
  x = x + 1
  wavfilename.append(ref[i])
  newlabels.append(label[i])
  newID.append(x)

In [27]:
len(newID)

350

In [28]:
dataframenew['new ID'] = newID
dataframenew['wav filename'] = wavfilename
dataframenew['labels'] = newlabels
dataframenew

,new ID,wav filename,labels
0,1,9.wav,1
1,2,10.wav,1
2,3,20.wav,0
3,4,2.wav,0
4,5,2.wav,0
...,...,...,...
345,346,/content/drive/MyDrive/ReCirCor/training_data/...,0
346,347,/content/drive/MyDrive/ReCirCor/training_data/...,0
347,348,/content/drive/MyDrive/ReCirCor/training_data/...,0
348,349,/content/drive/MyDrive/ReCirCor/training_data/...,1


In [29]:
dataframe

,filename,labels
0,1.wav,0
1,2.wav,0
2,3.wav,0
3,4.wav,0
4,5.wav,0
5,6.wav,1
6,7.wav,1
7,8.wav,1
8,9.wav,1
9,10.wav,1


In [30]:
dataframenew['labels'].value_counts()

1    187
0    163
Name: labels, dtype: int64

In [31]:
X_train, X_test, y_Train, y_Test = train_test_split(dataframenew['new ID'], dataframenew['labels'], train_size=0.8, shuffle=True)

In [32]:
X_train

18      19
308    309
178    179
338    339
128    129
      ... 
85      86
24      25
247    248
146    147
246    247
Name: new ID, Length: 280, dtype: object

In [33]:
X_test

269    270
83      84
180    181
49      50
109    110
      ... 
214    215
149    150
181    182
101    102
328    329
Name: new ID, Length: 70, dtype: object

In [34]:
#Computing Features - 

#Discrete Wavelet Transform
def computeDWTs(x):
    #approximation and detail coefficients
    res = np.array([])
    ca, cd = pywt.dwt(x, 'db8')
    #Compute mean, stdev, skew, kurtosis
    camean = np.mean(ca)
    res = np.hstack((res,camean))
    castdev = tstd(ca)
    res = np.hstack((res,castdev))
    caskew = skew(ca)
    res = np.hstack((res,caskew))
    cakurtosis = kurtosis(ca)
    res = np.hstack((res,cakurtosis))
    cdmean = np.mean(cd)
    res = np.hstack((res,cdmean))
    cdstdev = tstd(cd)
    res = np.hstack((res,cdstdev))
    cdskew = skew(cd)
    res = np.hstack((res,cdskew))
    cdkurtosis = kurtosis(cd)
    res = np.hstack((res, cdkurtosis))
    return res

#Computing the Spectral Entropy of the signal - Shannon Entropy of the Power Spectral Density (PSD) of the data
def SpectralEntropy(Signal, SampleRate, method = 'fft', nperseg = None, normalize = False):
    Signal = np.array(Signal)
    if method == 'fft':
      _, psd = periodogram(Signal, SampleRate)
    elif method == 'welch':
      _, psd = welch(Signal, SampleRate, nperseg=nperseg)
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    if normalize:
        se /= np.log2(psd_norm.size)
    return se

#Computing the Power Spectral Density(PSD) of signal - 
def computepsd(x):
    freq, psd = signal.welch(x)
    return psd

#Computing Mel-frequency cepstral coefficients, Gammatone frequency cepstral coefficients, RMS Energy, Zero-crossing rate, mean, variance, skew, and kurtosis of the signal
def computeMFCC(x, sr):
    mfccs = np.mean(librosa.feature.mfcc(y = x, sr = sr, n_mfcc = 13).T, axis = 0)
    return mfccs

def computeGFCC(file_name):
    gfcc = get_features(file=file_name, feature_names=["gfcc"])
    gfcc = np.array(gfcc['audio'][file_name]['features'])
    return gfcc

def computeRMSEn(x):
    rmsen = np.mean(librosa.feature.rms(x, frame_length=2048, hop_length=1024, center=True).T, axis = 0)
    return rmsen

def computeZCR(x):
    zcr = np.mean(librosa.feature.zero_crossing_rate(y = x, frame_length=2048, hop_length=1024).T, axis = 0)
    return zcr

def computestats(x):
#Mean, Variance, Skew, Kurtosis
    stats = []
    me = np.mean(x)
    stats.append(me)
    variance = np.var(x)
    stats.append(variance)
    sk = skew(x)
    stats.append(sk)
    kurt = kurtosis(x)
    stats.append(kurt)
    return stats

In [35]:
#Function to extract the required features
def extractfeatures(file_name, x, sr, *, mfcc: bool = False, dwt: bool = False, spectral_entropy: bool = False, psd: bool = False, gfcc: bool = False, rmse: bool = False, zcr: bool = False, stats: bool = False):
    finalfeatures = np.array([])
    if mfcc:
        mfccs = computeMFCC(x, sr)
        finalfeatures = np.hstack((finalfeatures, mfccs))
    if dwt:
        dwtr = computeDWTs(x)
        finalfeatures = np.hstack((finalfeatures, dwtr))
    if spectral_entropy:
        specen = SpectralEntropy(x, sr)
        finalfeatures = np.hstack((finalfeatures,specen))
    if psd:
        psds = computepsd(x)
        finalfeatures = np.hstack((finalfeatures, psds))
    if gfcc:
        gfccs = computeGFCC(file_name)
        finalfeatures = np.hstack((finalfeatures, gfccs))
    if rmse:
        rmses = computeRMSEn(x)
        finalfeatures = np.hstack((finalfeatures, rmses))
    if zcr:
        zcrs = computeZCR(x)
        finalfeatures = np.hstack((finalfeatures, zcrs))
    if stats:
        stats1 = computestats(x)
        finalfeatures = np.hstack((finalfeatures, stats1))
    return finalfeatures

In [36]:
#loads audio and extracts required features
def load_file_data(file_num, duration=15, sr = 4000):
    input_length=sr*duration
    notfeatures = []
    i = 0
    for file_no in file_num:
        try:
            #sound_file = folder + '/' + file_name
            sfile = int(file_no)
            sound_file = dataframenew['wav filename'][sfile-1]
            print ("load file ", i, sound_file)
            x, sr = librosa.load(sound_file,duration=duration,res_type='kaiser_fast')
            dur = librosa.get_duration(y=x, sr=sr)
            #Padding audio signal
            if (round(dur) < duration):
                print ("fixing audio length:", file_no, sound_file)
                y = librosa.util.fix_length(x, input_length)
            feature = extractfeatures(sound_file, x, sr, mfcc = True, dwt = True, psd = True, gfcc = True, rmse = True, zcr = True, stats = True)
            #dwt = True, psd = True, gfcc = True, rmse = True, zcr = True, stats = True
            notfeatures.append(feature)          
        except Exception as e:
            print(e)
            #notfeatures = np.asarray(features, dtype=np.float32)
        i = i+1 
    return notfeatures

In [37]:
x_train = []
x_test = []
x_train = load_file_data(X_train)
x_test = load_file_data(X_test)

load file  0 8.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 8.wav
load file  1 /content/drive/MyDrive/ReCirCor/training_data/training_data/40840_AV.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/40840_AV.wav
load file  2 4.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 4.wav
load file  3 /content/drive/MyDrive/ReCirCor/training_data/training_data/49561_TV.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49561_TV.wav
load file  4 10.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 10.wav
load file  5 1.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 1.wav
load file  6 10.wav

        
 Extracting features gfcc 

        
Computing

In [38]:
x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)

In [39]:
x_test.shape

(70, 200)

In [40]:
x_train.shape

(280, 200)

In [41]:
y_Train.dtype

dtype('int64')

In [42]:
y_Train = np.asarray(y_Train).astype(np.float32)

In [43]:
y_Test = np.asarray(y_Test).astype(np.float32)

In [44]:
traindataframe = pd.DataFrame(columns = ['x_train ID', 'labels'])
testdataframe = pd.DataFrame(columns = ['x_test ID', 'labels'])
traindataframe['x_train ID'] = X_train
testdataframe['x_test ID'] = X_test
traindataframe['labels'] = y_Train
testdataframe['labels'] = y_Test

In [45]:
testdataframe

,x_test ID,labels
269,270,0.0
83,84,1.0
180,181,1.0
49,50,1.0
109,110,0.0
...,...,...
214,215,0.0
149,150,1.0
181,182,1.0
101,102,0.0


In [46]:
xtrainwavfiles = list()
for i in X_test:
  num = int(i)
  xtrainwavfiles.append(dataframenew['wav filename'][num-1])

In [47]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.8 MB/s 


In [48]:
import kerastuner
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
def model_builder(hp):
  model = Sequential()
  hp_units1 = hp.Int('units_', min_value=2, max_value=64, step=4)
  model.add(tf.keras.layers.Dense(units = hp_units1, input_dim = x_train.shape[1]))

  # Tune the number of dense layers
  for i in range(hp.Int('num_layers', 1, 5)):
    
    # Tune the number of units in the each dense layer
    hp_units = hp.Int('units_'+str(i), min_value=2, max_value=64, step=4)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    
    # Tune the dropout rate in the each dense layer
    hp_dropout = hp.Float('rate', min_value=0.0, max_value=0.5, step=0.1)
    model.add(tf.keras.layers.Dropout(hp_dropout))
    
  # Add dense output layer
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  # Tune the learning rate for the optimizer
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

#tf.keras.metrics.Recall()
#tuner = BayesianOptimization(model_builder,
#max_trials=50,
#objective='val_accuracy',
#overwrite=True)

tuner = Hyperband(model_builder,
objective=kerastuner.Objective("val_accuracy", direction="max"),
max_epochs=50,
factor=3, overwrite = True)

tuner.search(x_train,y_Train, epochs=50, validation_split=0.2)
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)

Trial 90 Complete [00h 00m 06s]
val_accuracy: 0.6964285969734192

Best val_accuracy So Far: 0.8571428656578064
Total elapsed time: 00h 03m 39s


In [49]:
history = model.fit(x_train, y_Train, batch_size = 10, epochs=50, validation_split = 0.2, verbose = 1)

Epoch 1/50
23/23 [==============================] - 1s 12ms/step - loss: 343.1587 - accuracy: 0.5714 - val_loss: 174.4878 - val_accuracy: 0.5179
Epoch 2/50
23/23 [==============================] - 0s 4ms/step - loss: 296.8951 - accuracy: 0.5759 - val_loss: 110.9827 - val_accuracy: 0.6607
Epoch 3/50
23/23 [==============================] - 0s 4ms/step - loss: 165.1277 - accuracy: 0.5938 - val_loss: 171.2934 - val_accuracy: 0.6964
Epoch 4/50
23/23 [==============================] - 0s 4ms/step - loss: 242.3496 - accuracy: 0.6116 - val_loss: 89.2189 - val_accuracy: 0.8036
Epoch 5/50
23/23 [==============================] - 0s 4ms/step - loss: 121.5700 - accuracy: 0.6562 - val_loss: 35.3613 - val_accuracy: 0.6607
Epoch 6/50
23/23 [==============================] - 0s 4ms/step - loss: 231.0270 - accuracy: 0.5714 - val_loss: 24.7962 - val_accuracy: 0.4286
Epoch 7/50
23/23 [==============================] - 0s 4ms/step - loss: 145.1517 - accuracy: 0.6027 - val_loss: 65.6291 - val_accuracy: 0.

In [50]:
y_predict = np.argmax(model.predict(x_test), axis=-1)
y_pred = model.predict(x_test)
y_pred=np.argmax(y_pred, axis=1)

3/3 [==============================] - 0s 6ms/step


In [51]:
print("Classification Report : \n\n", classification_report(y_Test, y_pred))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.50      1.00      0.67        35
         1.0       0.00      0.00      0.00        35

    accuracy                           0.50        70
   macro avg       0.25      0.50      0.33        70
weighted avg       0.25      0.50      0.33        70



In [52]:
df = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred})
df

,Input,Wav filename,Real Values,Predicted Values
269,270,/content/drive/MyDrive/ReCirCor/training_data/...,0.0,0
83,84,23.wav,1.0,0
180,181,15.wav,1.0,0
49,50,21.wav,1.0,0
109,110,5.wav,0.0,0
...,...,...,...,...
214,215,2.wav,0.0,0
149,150,23.wav,1.0,0
181,182,17.wav,1.0,0
101,102,5.wav,0.0,0


In [53]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(x_train, y_Train)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [54]:
clf.score(x_test, y_Test)

0.9

In [55]:
y_pred1 = clf.predict(x_test)
print("Classification Report : \n\n", classification_report(y_Test, y_pred1))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.94      0.86      0.90        35
         1.0       0.87      0.94      0.90        35

    accuracy                           0.90        70
   macro avg       0.90      0.90      0.90        70
weighted avg       0.90      0.90      0.90        70



In [56]:
df1 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred1})
df1

,Input,Wav filename,Real Values,Predicted Values
269,270,/content/drive/MyDrive/ReCirCor/training_data/...,0.0,0.0
83,84,23.wav,1.0,1.0
180,181,15.wav,1.0,1.0
49,50,21.wav,1.0,1.0
109,110,5.wav,0.0,0.0
...,...,...,...,...
214,215,2.wav,0.0,0.0
149,150,23.wav,1.0,1.0
181,182,17.wav,1.0,1.0
101,102,5.wav,0.0,0.0


In [57]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [58]:
clf2 = LinearSVC(penalty='l2', loss='hinge')
clf2.fit(x_train, y_Train)
clf2.score(x_test, y_Test)

0.6714285714285714

In [59]:
y_pred2 = clf2.predict(x_test)
print("Classification Report : \n\n", classification_report(y_Test, y_pred2))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.93      0.37      0.53        35
         1.0       0.61      0.97      0.75        35

    accuracy                           0.67        70
   macro avg       0.77      0.67      0.64        70
weighted avg       0.77      0.67      0.64        70



In [60]:
df2 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred2})
df2

,Input,Wav filename,Real Values,Predicted Values
269,270,/content/drive/MyDrive/ReCirCor/training_data/...,0.0,0.0
83,84,23.wav,1.0,1.0
180,181,15.wav,1.0,1.0
49,50,21.wav,1.0,1.0
109,110,5.wav,0.0,0.0
...,...,...,...,...
214,215,2.wav,0.0,1.0
149,150,23.wav,1.0,1.0
181,182,17.wav,1.0,1.0
101,102,5.wav,0.0,0.0


In [61]:
clf4 = SGDClassifier(loss = 'hinge').fit(x_train, y_Train)

In [62]:
clf4.score(x_test, y_Test)

0.6142857142857143

In [63]:
y_pred4 = clf4.predict(x_test)
#y_pred=np.argmax(y_pred, axis=1)
print("Classification Report : \n\n", classification_report(y_Test, y_pred4))

Classification Report : 

               precision    recall  f1-score   support

         0.0       1.00      0.23      0.37        35
         1.0       0.56      1.00      0.72        35

    accuracy                           0.61        70
   macro avg       0.78      0.61      0.55        70
weighted avg       0.78      0.61      0.55        70



In [64]:
df4 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred4})
df4

,Input,Wav filename,Real Values,Predicted Values
269,270,/content/drive/MyDrive/ReCirCor/training_data/...,0.0,1.0
83,84,23.wav,1.0,1.0
180,181,15.wav,1.0,1.0
49,50,21.wav,1.0,1.0
109,110,5.wav,0.0,0.0
...,...,...,...,...
214,215,2.wav,0.0,1.0
149,150,23.wav,1.0,1.0
181,182,17.wav,1.0,1.0
101,102,5.wav,0.0,0.0
